# review outputs of analysis pipeline
## check_pipeline_outputs.ipynb

This script will open the .json config file used to run the analysis pipeline and the data saved out from the pipeline as .nc files. This notebook also has some examples of how to select data out of the xarray structures opened from .nc files. User only needs to enter a path to the .json config file used and the .nc files will be found from the save-path contained therein. Run this in the ```fmephys``` environment.

Last modified September 07, 2020

In [1]:
import os.path
import xarray as xr
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

from util.read_data import find

## user inputs

In [ ]:
# path to the folder of all xarrays saved out from the trial
parent_path = '/Users/dylanmartins/Dropbox//'
# the name of the trial to focus on
trial_name = '101420_G6H28P6LT_fm1'

## pipeline outputs

### ephys data

In [ ]:
ephys_path = os.path.join(parent_path, trial_name + '_ephys.json')
ephys_data = pd.read_json(ephys_path)

In [ ]:
ephys_data

### eye parameters and properties

In [ ]:
eye_nc_path = os.path.join(parent_path, (trial_name + 'Reye_current.nc'))
eye_data = xr.open_dataset(eye_nc_path)
eye_data

In [ ]:
# pull out the original DeepLabCut points
eye_dlc = eye_data['REYE_pts']

In [ ]:
# pull out the ellipse parameters
eye_ellipse = eye_data['REYE_ellipse_params']

In [ ]:
# pull out radius of eye at each possible degree
rfit = eye_data['Rradius_fit']

In [ ]:
# pull out radius of eye at each possible degree
rfit_conv = eye_data['Rradius_fit_conv']

In [ ]:
# pull out pupil rotation angle in degrees
shift_smooth = eye_data['Reye_pupil_rotation']
shift_smooth

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
rfit = pd.DataFrame(eye_rfit.values)

In [ ]:
np.shape(rfit)

In [ ]:
shift_smooth = eye_pupil_rot.values

In [ ]:
np.shape(shift_smooth)

In [ ]:
from scipy.optimize import curve_fit
from scipy import signal

In [ ]:
# sigmoid function
def curve_func(xval, a, b, c, d):
    return a+(b-a)/(1+10**((c-xval)*d))

# multiprocessing-ready fit to sigmoid function
def sigm_fit_mp(d):
    try:
        popt, pcov = curve_fit(curve_func, xdata=range(1,len(d)+1), ydata=d, p0=[100,200,10,0.5], bounds=([50, 100, 5, .05],[150, 250, 20, 5]), method='trf', xtol=10**-5)
        ci = np.sqrt(np.diagonal(pcov))
    except RuntimeError:
        popt = np.nan*np.zeros(4)
        ci = np.nan*np.zeros(4)
    return (popt, ci)

In [ ]:
eye_theta = eye_ellipse.sel(ellipse_params='theta')
eye_phi = eye_ellipse.sel(ellipse_params='phi')
eye_longaxis= eye_ellipse.sel(ellipse_params='longaxis')
eye_shortaxis = eye_ellipse.sel(ellipse_params='shortaxis')
eye_centX = eye_ellipse.sel(ellipse_params='X0')
eye_centY = eye_ellipse.sel(ellipse_params='Y0')

eyevid = cv2.VideoCapture('/Users/dylanmartins/Desktop/101420_G6H28P6LT_fm1_REYEdeinter.avi')
for step in range(0,5):
    try:
        # frame reading and black and white conversion
        eye_ret, eye_frame = eyevid.read()

        if not eye_ret:
            break

        eye_frame = cv2.cvtColor(eye_frame, cv2.COLOR_BGR2GRAY)

        # get ellisepe parameters for this time
        current_theta = eye_theta.sel(frame=step).values
        current_phi = eye_phi.sel(frame=step).values
        current_longaxis = eye_longaxis.sel(frame=step).values
        current_shortaxis = eye_shortaxis.sel(frame=step).values
        current_centX = eye_centX.sel(frame=step).values
        current_centY = eye_centY.sel(frame=step).values

        # some configuration
        meanr = 0.5 * (current_longaxis + current_shortaxis) # mean radius
        r = range(int(meanr - ranger), int(meanr + ranger)) # range of values over mean radius (meanr)
        pupil_edge = np.zeros([totalF, 360, len(r)]) # empty array that the calculated edge of the pupil will be put into

        rad_range = np.deg2rad(np.arange(360))
        # get cross-section of pupil at each angle 1-360 and fit to sigmoid
        for i in range(0, len(r)):
            pupil_edge[step,:,i] = eye_frame[((current_centY + r[i]*(np.sin(rad_range))).astype(int),(current_centX + r[i]*(np.cos(rad_range))).astype(int))]
        d = pupil_edge[step,:,:]
#         plt.figure()
#         plt.imshow(d, aspect='auto')
#         plt.show()
        params_output = []
        for n in range(360):
            params_output.append(sigm_fit_mp(d[n]))
        params = []; ci = []
        for vals in params_output:
            params.append(vals[0])
            ci.append(vals[1])
        params = np.stack(params); ci = np.stack(ci)

        fit_thresh = 1

        # extract radius variable from parameters
        rfit = params[:,2] - 1

        # if confidence interval in estimate is > fit_thresh pix, set to to NaN
        # then, remove if luminance goes the wrong way (e.g. from reflectance)
        for deg_th in range(0,360):
            rfit[deg_th] = np.where(ci[deg_th,2] > fit_thresh, np.nan, rfit[deg_th])
            rfit[deg_th] = np.where((params[deg_th,1] - params[deg_th,0]) < 0, np.nan, rfit[deg_th])
            
#         # interpolate because convolution will create large NaN holes
#         # is interpolation a good idea here? either way, the way this is done can be improved
#         interp_x = [item for sublist in np.argwhere(np.isnan(rfit)) for item in sublist]
#         interp_xp = [item for sublist in np.argwhere(~np.isnan(rfit)) for item in sublist]
#         interp_fp = rfit[~np.isnan(rfit)]

        try:
#             rfit_interp_vals = np.interp(interp_x, interp_xp, interp_fp)
#             # replace values in rfit_interp if they were NaN with the values found in interpolation
#             rfit_interp = rfit; j=0
#             for i in range(0,len(rfit_interp)):
#                 if np.isnan(rfit_interp[i]):
#                     rfit_interp[i] = rfit_interp_vals[j]
#                     j = j + 1

            # median filter
            rfit_interp = signal.medfilt(rfit,3)

            # subtract baseline because our points aren't perfectly centered on ellipse
            filtsize = 30
            rfit_conv = rfit - np.convolve(rfit_interp, np.ones(filtsize)/filtsize, mode='same')
            # edges have artifact from conv, so set to NaNs
            # could fix this by padding data with wraparound at 0 and 360deg before conv
            # the astropy package can do this with the convolution.convolve package
            # TO DO: test and impliment wraparound convolution with astropy function convolve
            rfit_conv[range(0,int(filtsize/2+1))] = np.nan
            rfit_conv[range((len(rfit_conv)-int(filtsize/2-1)),len(rfit_conv))] = np.nan

        except ValueError: # in case every value in rfit is NaN
            rfit_conv = np.empty(np.shape(rfit_conv)) # make an rfit_conv with the shape of the last one
    except (KeyError, ValueError) as e:
        key_error_count = key_error_count + 1
        rfit_conv = np.empty(np.shape(rfit_conv))

In [ ]:
# calculates xcorr ignoring NaNs without altering timing
# adapted from /niell-lab-analysis/freely moving/nanxcorr.m
# always normalizes inputs, assuming 'coeff' flag in matlab code is given
def nanxcorr(x, y, maxlag=25):
    lags = range(-maxlag, maxlag)
    cc = []
    for i in range(0,len(lags)):
        # shift data
        yshift = np.roll(y, lags[i])
        # get index where values are usable in both x and yshift
        use = ~pd.isnull(x + yshift)
        # some restructuring
        x_arr = np.asarray(x, dtype=object); yshift_arr = np.asarray(yshift, dtype=object)
        x_use = x_arr[use]; yshift_use = yshift_arr[use]
        # normalize
        x_use = (x_use - np.mean(x_use)) / (np.std(x_use) * len(x_use))
        yshift_use = (yshift_use - np.mean(yshift_use)) / np.std(yshift_use)
        # get correlation
        cc.append(np.correlate(x_use, yshift_use))
    cc_out = np.hstack(np.stack(cc))
    return cc_out, lags

In [ ]:
n = np.size(rfit_conv.values, 0)
pupil_update = rfit_conv.values
total_shift = np.zeros(n); peak = np.zeros(n)
c = total_shift
template = np.nanmean(rfit_conv.values, 0)

In [ ]:
from astropy.convolution import convolve

In [ ]:
print('doing iterative fit on frames to find alignment for each frame')
for rep in tqdm(range(0,12)):

    # for each frame, get correlation, and shift
    for frame_num in range(0,n):
        try:
            xc, lags = nanxcorr(template, pupil_update[frame_num,:], 10)
            c[frame_num] = np.amax(xc) # value of max
            peaklag = np.argmax(xc) # position of max
            peak[frame_num] = lags[peaklag]
            total_shift[frame_num] = total_shift[frame_num] + peak[frame_num]
            pupil_update[frame_num,:] = np.roll(pupil_update[frame_num,:], int(peak[frame_num]))
        except ZeroDivisionError:
            total_shift[frame_num] = np.nan
            pupil_update[frame_num,:] = np.nan
            
#     if config['save_figs'] is True:
#         # plot template with pupil_update for each iteration of fit
#         plt.figure()
#         plt.title('pupil_update of rep='+str(rep)+' in iterative fit')
#         plt.plot(template, 'k--', alpha=0.8)
#         plt.plot(pupil_update.T, alpha=0.2)
#         pdf.savefig()
#         plt.close()

#         # histogram of correlations
#         plt.figure()
#         plt.title('correlations of rep='+str(rep)+' in iterative fit')
#         plt.hist(c)
#         pdf.savefig()
#         plt.close()

In [ ]:
plt.plot(template, 'k--', alpha=0.8)
plt.plot(pupil_update.T, alpha=0.2)
plt.ylim(0,25)

In [ ]:
np.rad2deg(total_shift)

In [ ]:
np.shape(pupil_update)

In [ ]:
frame_num = 200
xc, lags = nanxcorr(template, pupil_update[frame_num,:], 10)
c[frame_num] = np.amax(xc)
peaklag = np.argmax(xc)
peak[frame_num] = lags[peaklag]
total_shift[frame_num] = total_shift[frame_num] + peak[frame_num]
pupil_update[frame_num,:] = np.roll(pupil_update[frame_num,:], int(peak[frame_num]))

In [ ]:
c[frame_num]

In [ ]:
peak[frame_num]

In [ ]:
plt.plot(xc)

In [ ]:
from scipy.io import savemat

savemat('/Users/dylanmartins/Desktop/test101420_pupil_update_frame100.mat', {'pupil_update':pupil_update[100,:]})


In [ ]:
np.shape(xc)

In [ ]:
c[frame_num] = np.amax(xc) # value of max
peaklag = np.argmax(xc) # position of max
peak[frame_num] = lags[peaklag]
total_shift[frame_num] = total_shift[frame_num] + peak[frame_num]
pupil_update[frame_num,:] = np.roll(pupil_update[frame_num,:], int(peak[frame_num]))

In [ ]:
total_shift[np.mean(rfit_conv,1) > 25] = np.nan

In [ ]:
plt.plot(total_shift)

In [ ]:
plt.plot(rfit_conv.T)

In [ ]:
xc

In [ ]:
win = 3
shift_nan = -total_shift
shift_nan[c < 0.2] = np.nan # started at [c < 0.4], is it alright to change this? many values go to NaN otherwise
shift_nan[shift_nan > 0.75] = np.nan; shift_nan[shift_nan < -0.75] = np.nan
shift_smooth = convolve(shift_nan, np.ones(win)/win, boundary='wrap') # convolve using astopy.convolution.convolve, which should work like nanconv by interpolating over nans as appropriate
shift_smooth = shift_smooth - np.nanmedian(shift_smooth)
shift_nan = shift_nan - np.nanmedian(shift_nan)

In [ ]:
plt.plot(total_shift)

In [ ]:
c

In [ ]:
plt.plot(shift_smooth)
plt.ylim(-1,1)

In [ ]:
plt.plot(total_shift)
plt.ylim(-1,1)

In [ ]:
shift_smooth

In [ ]:
shift_xr = xr.DataArray(shift_smooth)

In [ ]:
shift_xr

In [ ]:
plt.plot(ci[:,0])
# plt.ylim(0,5)

In [ ]:
plt.plot(ci[:,1])

In [ ]:
plt.plot(rfit)

In [ ]:
plt.plot(rfit_conv)

In [ ]:
plt.hist(rfit_conv, bins=100)

In [ ]:
rfit

In [ ]:
rfit_conv

In [ ]:
eye_theta = eye_ellipse.sel(ellipse_params='theta')
eye_phi = eye_ellipse.sel(ellipse_params='phi')
eye_longaxis= eye_ellipse.sel(ellipse_params='longaxis')
eye_shortaxis = eye_ellipse.sel(ellipse_params='shortaxis')
eye_centX = eye_ellipse.sel(ellipse_params='X0')
eye_centY = eye_ellipse.sel(ellipse_params='Y0')

ranger = 10
rad_range = np.deg2rad(np.arange(360))

eyevid = cv2.VideoCapture('/Users/dylanmartins/Desktop/101420_G6H28P6LT_fm1_REYEdeinter.avi')
vidsavepath = os.path.join('/Users/dylanmartins/Desktop/', str(trial_name + '_pupil_rotation_REYE_4.avi'))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
vidout = cv2.VideoWriter(vidsavepath, fourcc, 60.0, (int(eyevid.get(cv2.CAP_PROP_FRAME_WIDTH)), int(eyevid.get(cv2.CAP_PROP_FRAME_HEIGHT))))

totalF = int(eyevid.get(cv2.CAP_PROP_FRAME_COUNT))

# shift_smooth = np.squeeze(shift_smooth)

print('plotting pupil rotation on eye video')
for step in tqdm(np.arange(totalF)):    
    eye_ret, eye_frame = eyevid.read()

    if not eye_ret:
        break

#     eye_frame = cv2.cvtColor(eye_frame, cv2.COLOR_BGR2GRAY)

    # get ellisepe parameters for this time
    current_theta = eye_theta.sel(frame=step).values
    current_phi = eye_phi.sel(frame=step).values
    current_longaxis = eye_longaxis.sel(frame=step).values
    current_shortaxis = eye_shortaxis.sel(frame=step).values
    current_centX = eye_centX.sel(frame=step).values
    current_centY = eye_centY.sel(frame=step).values

    # plot the ellipse edge
    rmin = 0.5 * (current_longaxis + current_shortaxis) - ranger
    for deg_th in range(0,360):
        rad_th = rad_range[deg_th]
        edge_x = np.round(current_centX+(rmin + rfit.iloc[step,deg_th])*np.cos(rad_th))
        edge_y = np.round(current_centY+(rmin + rfit.iloc[step,deg_th])*np.sin(rad_th))
        if pd.isnull(edge_x) is False and pd.isnull(edge_y) is False:
            eye_frame = cv2.circle(eye_frame, (int(edge_x),int(edge_y)), 1, (235,52,155), thickness=-1)

#     # plot the rotation of the eye as a vertical line made up of many circles
#     for d in range(-40,40):
#         rot_x = np.round(current_centX + d * (np.cos(shift_smooth[step])))
#         rot_y = np.round(current_centY + d * (np.sin(shift_smooth[step])))
#         if pd.isnull(rot_x) is False and pd.isnull(rot_y) is False:
#             eye_frame = cv2.circle(eye_frame, (int(rot_x),int(rot_y)),1,(255,255,255),thickness=-1)
#         else:
#             e_count = e_count + 1

#     # plot the center of the eye on the frame as a larger dot than the others
#     if pd.isnull(current_centX) is False and pd.isnull(current_centY) is False:
#         eye_frame = cv2.circle(eye_frame, (int(current_centX),int(current_centY)),6,(0,0,255),thickness=-1)

    vidout.write(eye_frame)

vidout.release()

In [ ]:
plt.hist(rfit.iloc[100,:], bins=100)

In [ ]:
0.5 * np.mean(eye_longaxis) + np.mean(eye_shortaxis)

In [ ]:
for i in rfit.iloc[100,:]:
    print(i)

In [ ]:
np.round(current_centX+(rfit.iloc[100,100])*np.cos(rad_th))

In [ ]:
rmin

In [ ]:
np.rad2deg(np.sin(shift_smooth[step]))

In [ ]:
np.round(current_centY + d * np.rad2deg(np.sin(shift_smooth[step])))

In [ ]:
current_centY

In [ ]:
e_count

In [ ]:
int(rot_y)

In [ ]:
shift_smooth

In [ ]:
np.rad2deg(shift_smooth)

In [ ]:
np.round(current_centX + d1 * np.cos(np.deg2rad(shift_smooth[current_time]+90)))

In [ ]:
np.round(current_centY + d1 * np.sin(np.deg2rad(shift_smooth[current_time]+90)))

In [ ]:
np.cos(np.deg2rad(shift_smooth[current_time]+90))

In [ ]:
np.sin(np.deg2rad(shift_smooth[current_time]+90))

In [ ]:
rmin+rfit.iloc[step,deg_th]

In [ ]:
np.array(rfit.iloc[100,:])

In [ ]:
deg_th

In [ ]:
shift_smooth

In [ ]:
rmin

In [ ]:
print(int(np.round(94.08752408)))

In [ ]:
int(np.round(current_centY+(rmin+rfit.iloc[step,deg_th])*np.sin(rad_th)))

In [ ]:
rfit.isel(frame=100).values

In [ ]:
eye_theta = eye_ellipse.sel(ellipse_params='theta')
eye_phi = eye_ellipse.sel(ellipse_params='phi')
eye_longaxis= eye_ellipse.sel(ellipse_params='longaxis')
eye_shortaxis = eye_ellipse.sel(ellipse_params='shortaxis')
eye_centX = eye_ellipse.sel(ellipse_params='X0')
eye_centY = eye_ellipse.sel(ellipse_params='Y0')

In [ ]:
shift_smooth1 = xr.DataArray(shift_smooth, dims=['frame'])

In [ ]:
print(np.min(np.rad2deg(shift_smooth1)))
print(np.max(np.rad2deg(shift_smooth1)))

In [ ]:
eyevid = cv2.VideoCapture('/Users/dylanmartins/Desktop/101420_G6H28P6LT_fm1_REYEdeinter.avi')
fourcc = cv2.VideoWriter_fourcc(*'XVID')

vidsavepath = '/Users/dylanmartins/Desktop/101420_pupil_rotation_plot_test_8_thresh.avi'

vidout = cv2.VideoWriter(vidsavepath, fourcc, 60.0, (int(eyevid.get(cv2.CAP_PROP_FRAME_WIDTH)), int(eyevid.get(cv2.CAP_PROP_FRAME_HEIGHT))))

ranger = 10
rad_range = np.deg2rad(np.arange(360))
totalF = int(eyevid.get(cv2.CAP_PROP_FRAME_COUNT))

print('plotting pupil rotation on eye video')
for step in tqdm(np.arange(totalF)):
    
    eye_ret, eye_frame = eyevid.read()

    if not eye_ret:
        break

    # get ellisepe parameters for this time
    current_theta = eye_theta.sel(frame=step).values
    current_phi = eye_phi.sel(frame=step).values
    current_longaxis = eye_longaxis.sel(frame=step).values
    current_shortaxis = eye_shortaxis.sel(frame=step).values
    current_centX = eye_centX.sel(frame=step).values
    current_centY = eye_centY.sel(frame=step).values

    # plot the ellipse edge
    rmin = 0.5 * (current_longaxis + current_shortaxis) - ranger
    for deg_th in range(0,360):
        rad_th = rad_range[deg_th]
        edge_x = np.round(current_centX+(rmin+rfit.isel(frame=step,deg=deg_th).values)*np.cos(rad_th))
        edge_y = np.round(current_centY+(rmin+rfit.isel(frame=step,deg=deg_th).values)*np.sin(rad_th))
        if pd.isnull(edge_x) is False and pd.isnull(edge_y) is False:
            eye_frame = cv2.circle(eye_frame, (int(edge_x),int(edge_y)), 1, (235,52,155), thickness=-1)

    # plot the rotation of the eye as a vertical line made up of many circles
    for d in np.linspace(-0.5,0.5,100):
        rot_x = np.round(current_centX + d*(np.rad2deg(np.cos(shift_smooth1.isel(frame=step).values))))
        rot_y = np.round(current_centY + d*(np.rad2deg(np.sin(shift_smooth1.isel(frame=step).values))))
        if pd.isnull(rot_x) is False and pd.isnull(rot_y) is False:
            eye_frame = cv2.circle(eye_frame, (int(rot_x),int(rot_y)),1,(255,255,255),thickness=-1)

    # plot the center of the eye on the frame as a larger dot than the others
    if pd.isnull(current_centX) is False and pd.isnull(current_centY) is False:
        eye_frame = cv2.circle(eye_frame, (int(current_centX),int(current_centY)),3,(0,255,0),thickness=-1)

    vidout.write(eye_frame)

vidout.release()

In [ ]:
# histogram of correlations
plt.title('correlations of rep='+str(rep)+' in iterative fit')
plt.hist(c, bins=300)
plt.xlim(-2.5,2.5)

In [ ]:
plt.plot(rfit_conv.T, alpha=0.3)
plt.plot(np.mean(rfit_conv.T, 1), 'b--')
plt.title('convolved rfit for all trials, mean in blue')
# plt.ylim([-4,4])

In [ ]:
timepoint_corr_rfit = pd.DataFrame(rfit_conv.values).T.corr()
fig, ax = plt.subplots()
im = ax.imshow(timepoint_corr_rfit)
ax.set_title('correlation of radius fit across timepoints')
ax.set_xticks(np.arange(len(timepoint_corr_rfit)))
ax.set_yticks(np.arange(len(timepoint_corr_rfit)))
ax.set_xticklabels(np.linspace(1,25, len(timepoint_corr_rfit)+1))
ax.set_yticklabels(np.linspace(1,25, len(timepoint_corr_rfit)+1))
fig.colorbar(im, ax=ax)

In [ ]:
np.linspace(-10,10,100)

In [ ]:
plt.plot(shift_smooth1)
plt.ylim(-0.8,0.5)

In [ ]:
current_centX + d * np.cos(np.deg2rad(shift_smooth1.isel(frame=step).values))

In [ ]:
np.rad2deg(shift_smooth1.isel(frame=50).values)

In [ ]:
shift_smooth1.isel(frame=step).values

In [ ]:
shift_smooth

In [ ]:
current_centX = eye_centX.sel(frame=100).values
current_centX

In [ ]:
for deg in range(0,360):
    print(np.round(current_centX+(rmin+rfit.isel(frame=-1,deg=deg).values)*np.cos(rad_th)))

In [ ]:
(rmin+rfit.isel(frame=0,deg=deg).values)*np.cos(rad_th)

In [ ]:
current_centX

## example selection and indexing

In [ ]:
# here, we'll grab phi from frame 0 to 100 and plot it for a given trial
phi1d = eye_ellipse.sel(trial='112619_J463b_cricket2clip0',ellipse_params='phi').isel(frame=slice(0,100)).plot()

In [ ]:
# next, we'll plot the angle of rotation of the pupil for those same frames
puprot1d = eye_pupil_rot.sel(trial='112619_J463b_cricket2clip0').isel(frame=slice(0,20)).plot()